In [1]:
import openrec

In [2]:
import pkg_resources
pkg_resources.get_distribution('openrec').version

'0.2.5'

### TF version 2

In [3]:
from openrec.tf2.metrics.ranking_metrics import Recall, NDCG, AUC

In [4]:
import tensorflow as tf

In [5]:
tf.__version__

'2.5.0'

In [6]:
import pandas as pd
import numpy as np

df_pred = pd.read_csv('./pred.csv')
df_test = pd.read_csv('./test.csv')

df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

df_pred_new.fillna(0, inplace=True)

df_pred_new.relevance.value_counts()

0.0    51516
4.5     1840
5.0     1304
Name: relevance, dtype: int64

In [7]:
all_items = np.unique(np.hstack(
    [df_pred_new.item_id.unique(), df_test.item_id.unique()]
))

In [8]:
all_users = df_test.user_id.unique()

In [9]:
user_dict = dict(zip(all_users, range(len(all_users))))
item_dict = dict(zip(all_items, range(len(all_items))))

In [10]:
user_pred_np = np.zeros((len(all_users), len(all_items)))
user_pos_mask_np = np.zeros((len(all_users), len(all_items)), dtype=bool)

for user_id, item_id, score in zip(df_pred_new.user_id, df_pred_new.item_id, df_pred_new.score):
    user_pred_np[user_dict[user_id]][item_dict[item_id]] = score

for user_id, item_id, score in zip(df_test.user_id, df_test.item_id, df_test.relevance):
    user_pos_mask_np[user_dict[user_id]][item_dict[item_id]] = True

In [11]:
arr_inds = user_pred_np.argsort(axis=1)
arr_inds = np.flip(arr_inds, axis=1)

In [12]:
user_pred_np = np.take_along_axis(user_pred_np, arr_inds, axis=1)
user_pos_mask_np = np.take_along_axis(user_pos_mask_np, arr_inds, axis=1)

In [13]:
user_excl_mask = tf.constant(np.zeros_like(user_pos_mask_np, dtype=bool))
user_pred = tf.constant(user_pred_np.astype(np.float32), dtype=tf.float32)
user_pos_mask = tf.constant(user_pos_mask_np, dtype=tf.bool)

In [14]:
tf.math.reduce_mean(
    Recall(pos_mask=user_pos_mask, pred=user_pred, excl_mask=user_excl_mask, at=[20])
)

<tf.Tensor: shape=(), dtype=float32, numpy=0.096321315>

In [15]:
tf.math.reduce_mean(
    NDCG(pos_mask=user_pos_mask, pred=user_pred, excl_mask=user_excl_mask, at=[20])
)

<tf.Tensor: shape=(), dtype=float32, numpy=0.4626283>

In [16]:
user_auc = AUC(pos_mask=user_pos_mask[:, :20], pred=user_pred[:, :20], excl_mask=user_excl_mask[:, :20])
user_auc = tf.where(tf.math.is_nan(user_auc), tf.zeros_like(user_auc), user_auc)
tf.math.reduce_mean(user_auc)

<tf.Tensor: shape=(), dtype=float32, numpy=0.28317448>

### TF version 1

In [17]:
from openrec.tf1.utils.evaluators import *

In [18]:
def func(series):
    res = []
    for i, x in enumerate(series):
        if x > 0:
            res.append(i)
    return res

true_pos = df_pred_new.groupby('user_id')['relevance'].apply(func).tolist()

In [19]:
np.mean([Precision(precision_at=[20]).compute(x, None) for x in true_pos])

0.057519209659714604